In [1]:
import pandas as pd
#load wiki data for london
wiki_data = pd.read_csv('../../data/data_wiki_clustering/wikipedia_geotags_112020.csv')

In [2]:
#convert to geopandas dataframe
import geopandas as gpd
from geopandas import GeoDataFrame
kc_wiki_gpd = gpd.GeoDataFrame(wiki_data, geometry = gpd.points_from_xy(wiki_data.gt_lon, 
                                                                        wiki_data.gt_lat, 
                                                                        crs="EPSG:4326"))
#convert projection of wiki_gpd
kc_wiki_gpd = kc_wiki_gpd.to_crs('EPSG:27700')


In [8]:
#open  Kensington and Chelsea shapefile
lon_borough = gpd.read_file('../../data/boundary_london/London_Borough_Excluding_MHW.shp')
kc_borough = lon_borough[lon_borough['NAME'] == 'Kensington and Chelsea']

In [9]:
kc_borough

,NAME,GSS_CODE,HECTARES,NONLD_AREA,ONS_INNER,SUB_2009,SUB_2006,geometry
23,Kensington and Chelsea,E09000020,1238.379,25.994,T,None,None,"POLYGON ((526219.700 176948.000, 526208.800 17..."


In [ ]:
#extract wiki for kc only
kc_wiki_clip = gpd.overlay(kc_wiki_gpd, 
                           kc_borough, 
                           how='intersection')
#remove non-primary wiki pages gt_primary = 0
kc_wiki_clip = kc_wiki_clip[kc_wiki_clip.gt_primary != 0]

In [ ]:
pip install wikipedia-api
#call page summary data
page_title_column = "page_title"
#kc_wiki_clip["page_summary"] = ""
columns = ["page_title", "page_summary"]
page_summary_data = pd.DataFrame(columns=columns)
import wikipediaapi
for each_title in kc_wiki_clip[page_title_column]:
    #this_page =kc_wiki_clip[kc_wiki_clip[page_title_column] == each_title] 
    print("Working on %s:" % (each_title))
    wiki_wiki = wikipediaapi.Wikipedia('en')
    each_title_py = wiki_wiki.page(each_title)
    #this_page_py = wiki_wiki.page(each_title)
    page_summary = each_title_py.summary
    page_summary_data = page_summary_data.append({"page_title": each_title, 
                                                  "page_summary": page_summary},
                                                 ignore_index=True)

In [ ]:
kc_wiki_with_summary = kc_wiki_clip.merge(page_summary_data, 
                                          on = "page_title", 
                                          how = "left")
kc_wiki_with_summary = kc_wiki_with_summary.drop_duplicates() 
kc_wiki_with_summary.to_csv('../../data/data_wiki_clustering/kc_wiki_with_summary.csv')